# This notebook is intended to be used as a work-in-progress rapid prototyping development environment for the recreation of the proposed method of anomaly detection for IDS purposes from [this paper](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8751972)

### Use the SMOTE technique to balance the dataset (which essentially gives the classifier more data from outliers, thus enhancing its ability to classify such data):

### Firstly, we are going to do some feature extraction, in order to provide the model with vectorized data: